------------------------------------------------------------
### Course Project PCQI: Time-Dependent processes with Neural Quantum States
------------------------------------------------------------
This notebook is part of the course project for PCQI 2022.
By Pim Veefkind (XXXXXXX) & Thomas Rothe (1930443)

Set-up of basic NQS framework & implementation of time evolution + evaluation of expectations


---------------------------------------------------
#### Basic set-up of NQS:

In [1]:
import numpy as np
import pickle
#import scipy.sparse as sp
np.random.seed(12)

def sigmoid(X):
    return 1./(np.exp(X)+1)

def LnRMBWavefunction(W,a,c,V):
    #
    # Golden rule of numerics: avoid exponentials.
    # Use ln's instead.
    #
    Wsummed = 0
    LnPreFactor = 0
    L = V.shape[0]
    for s in range(L):
        Wsummed = Wsummed + W[:,s]*V[s]
        LnPreFactor = LnPreFactor - a[s]*V[s]
    
    # Difference between bits 0 and 1 and spins -1 and 1
    LnPrePreFactor = np.sum(a)/2 + np.sum(c)/2+np.sum(W)/4
    AngleFactor = np.prod(1+np.exp(-c - Wsummed))
    LnPsiRMB = LnPrePreFactor + LnPreFactor + np.log(AngleFactor)
    return LnPsiRMB

#### MC (Metropolis) Simulation:

In [7]:
def MetropolisCycle(W,a,c,Vt):
    rejectvalue = 0   
    LnPsiOld = LnRMBWavefunction(W,a,c,Vt)
    #
    # Flip a random spin
    # 
    L = Vt.shape[0] 
    site = np.random.randint(L)
    Vt[site] = - Vt[site] +1
    LnPsiNew = LnRMBWavefunction(W,a,c,Vt)
    #
    acceptanceratio = np.exp(np.real(np.conj(LnPsiNew)+LnPsiNew-np.conj(LnPsiOld)-LnPsiOld))
    #if acceptanceratio #MISSING INEQUALITY SIGN# 1:
    if acceptanceratio >= 1:
        return Vt,rejectvalue
    else:
        p = np.random.rand()
        #if p #MISSING INEQUALITY SIGN# acceptanceratio:
        if p >= acceptanceratio:
            rejectvalue = 1
            Vt[site] = - Vt[site] + 1
            
        return Vt,rejectvalue

def MetropolisSamp(W,a,c,V,k):
    #
    # Burn-in to get rid of initial condition dependence
    #
    rejections = 0
    rejectvalue = 0
    burn_in = 10000

    for z in range(burn_in):
        Vt = V
        V,rejectvalue = MetropolisCycle(W,a,c,Vt)
        rejections = rejections + rejectvalue
    
    print('Percentage Rejections in Burn-in: %.2f %%' %(rejections/burn_in*100))
    #
    #
    # We collect the full sequence of spin configurations V
    # Together they form a efficient short representation of the full distribution
    # 
    rejections = 0
    rejectvalue = 0
    Vensemble = np.copy(V)
    L = np.shape(V)[0]
    for z in range(k):
        # initiate sweep, i.e. cycle over # visible spins between appending
        for zz in range(L):
            V,rejectvalue = MetropolisCycle(W,a,c,V)
        Vensemble = np.append(Vensemble,V)
        rejections = rejections + rejectvalue
    
    prctrej = 100*rejections/k
    #print('Percentage Rejections in Ensemble: %.1f %% (%i/%i)' %(prctrej,rejections,k))
    Vensemble_reshape = Vensemble.reshape((k+1,L))
    # print(Vensemble_reshape)
    return Vensemble_reshape, prctrej 

#### Optimization of (time-independent) cost:

In [3]:
def Elocal(J,B,W,a,c,V):
    #
    # Computing the wavefunction for state V
    #
    L = V.shape[0]
    LnPsi = LnRMBWavefunction(W,a,c,V)
    LnPsiBar = np.conj(LnPsi)
    #
    # Computing the energy for state V
    # First the Ising term
    #
    Vshift = np.array([V[(i+1)%L] for i in range(L)])
    One = np.ones(L)
    ElocalJ = -J*(np.sum((2*V-One)*(2*Vshift-One)))
    #
    # Next the magnetic term -B\sum_i \sigma^x_i
    # Because this is not diagonal on the
    # states, we compute 
    # <V|EB|Psi> instead
    # The action of Sigma^x_i is
    # to flip the spin on site i:
    # i.e. map V[i] to -V[i]+1
    #
    EBlocalPsi = 0
    for i in range(L):
        V[i] = -V[i]+1
        EBlocalPsi = EBlocalPsi - B*np.exp(LnRMBWavefunction(W,a,c,V)-LnPsi) #Compare flipped with unflipped (sigma_x applied)
        V[i] = -V[i]+1
    
    ElocalPsi = ElocalJ + EBlocalPsi
    
    return ElocalPsi

    

def WeightUpdateSmoothed(J,B,W,a,c,Vensemble,lrate,ep):   
    # 
    # Now we will see the advantage of the Monte Carlo sampling.
    # Instead of summing over all spin configurations, 
    # we sum only over the ones generated by the MCMC Metropolis/Gibbs routine
    # to compute expectation values:
    # <Psi|Operator|Psi> = \sum_{all S,S'} <Psi|S><S|Operator|S'><S'|Psi>
    # is approximated by
    # <Psi|Operator|Psi> \simeq \sum_{Gibbs S,S'} <Psi|S><S|Operator|S'><S'|Psi>
    # For L large dim(S)=2^L, whereas we only need a finite number of Gibbs samples
    # So this will help greatly at large L
    #
    LenEnsemb = Vensemble.shape[0]
    L = Vensemble.shape[1]
    H = c.shape[0]
    #
    # Initializing for ensemble Exp(ectation)Val(ue)
    #
    LnNormPsi = 0
    EExpVal = 0
    ElocalExpVal = 0
    ElocalVExpVal = 0
    ElocalHExpVal = 0
    ElocalWExpVal = 0
    VExpVal = 0
    HExpVal = 0
    WExpVal = 0
    agradientEExpVal = 0
    cgradientEExpVal = 0
    WgradientEExpVal = 0
    derivsExpVal = 0
    moment2ExpVal = 0
    for l in range(LenEnsemb):
        V = Vensemble[l]
        #
        # V now labels a particular state
        #
        # Computing the energy for state V
        #
        ElocalPsi = Elocal(J,B,W,a,c,V)
        #
        # Next we compute 
        # <V|EV|V> = Elocal*V
        # <V|EH|V> = <Esigmoid(WV+c)> =Elocal*
        # <V|EHV|V> = <EVsigmoid(WV+c)>
        #
        ElocalVPsi = ElocalPsi*V 
        ElocalHPsi = ElocalPsi*sigmoid(c + np.matmul(W,V))  #sigmoid = current h vector
        ElocalWPsi = ElocalPsi*np.outer(sigmoid(c + np.matmul(W,V)),V)
        # 
        # Next we compute 
        # <V>
        # <H>
        # <HV>
        #
        derivs = np.concatenate((V,np.real(sigmoid(c+np.matmul(W,V))),np.real(np.outer(sigmoid(c+np.matmul(W,V)),V)).reshape(L*H)))
        #
        # Matrix of conj.derivs \times derivs
        #
        moment2 = np.outer(np.conj(derivs),derivs)
        #
        # Computing ensemble averages (uniform distrib. over all sampled configs)
        #
        ElocalExpVal = ElocalExpVal + ElocalPsi/LenEnsemb
        ElocalVExpVal = ElocalVExpVal + np.real(ElocalVPsi)/(LenEnsemb)
        ElocalHExpVal = ElocalHExpVal + np.real(ElocalHPsi)/(LenEnsemb)
        ElocalWExpVal = ElocalWExpVal + np.real(ElocalWPsi)/(LenEnsemb)
        derivsExpVal = derivsExpVal + derivs/LenEnsemb
        moment2ExpVal = moment2ExpVal + moment2/LenEnsemb
        #
        
    # Statistical local gradients, ignoring the quantum mechanical term
    #
    VExpVal = derivsExpVal[:L]
    HExpVal = derivsExpVal[L:L+H]
    WExpVal = derivsExpVal[L+H:].reshape(H,L)
    agradientEStat = - ElocalVExpVal + ElocalExpVal*VExpVal
    cgradientEStat = - ElocalHExpVal + ElocalExpVal*HExpVal
    WgradientEStat = - ElocalWExpVal + ElocalExpVal*WExpVal
    #
    # Computing metric on Probability space
    #
    #   - Cartesian metric as default
    #
    S_kkCartesian = np.diag(np.ones(L*H+L+H))
    #
    #   - Sorella version
    #
    S_kkSorella = moment2ExpVal - np.outer(np.conj(derivsExpVal),derivsExpVal)
    #
    #   - Regulator necessary to ensure inverse exists
    #
    lreg = np.max(np.array([100*(0.9)**ep,0.01]))  
    S_kkSorellaReg = S_kkSorella + lreg * np.diag(np.diag(S_kkCartesian))
    #
    #S_kk = S_kkCartesian
    S_kk = S_kkSorellaReg #Sorella = use variance in parameters/their derivates to adjust learning rate individually (per parameter type, per parameter)!
    #print("Average eigenvalue Sorella matrix:", np.trace(S_kk)/(L*H+L+H))
    #print(S_kk[L+H:,L+H:])
    #
    agrad = np.copy(agradientEStat)
    cgrad = np.copy(cgradientEStat)
    Wgrad = np.copy(WgradientEStat)
    #
    # Print out average length-squared of gradients as diagnostic
    # (finding good initial guess of model parameters manually)
    #
    GradAAbsSq = np.real(np.inner(np.conj(agrad),agrad))/L
    GradCAbsSq = np.real(np.inner(np.conj(cgrad),cgrad))/H
    GradWAbsSq = np.real(np.sum(np.conj(Wgrad)*Wgrad))/(L*H)
    print('\rGradient absval-squared: a: %.4f, c: %.4f, W: %.4f. ' %(GradAAbsSq,GradCAbsSq,GradWAbsSq), end='')
    #
    #
    Wgradtemp = Wgrad.reshape(L*H)
    paras = np.concatenate((a,c,W.reshape(L*H)))
    gradE = np.conj(np.concatenate((agrad,cgrad,Wgradtemp)))
    #
    deltaparas = lrate * np.einsum('ij,j->i',np.linalg.inv(S_kk),gradE) #Learning rate in metric x gradient
    paras = paras - deltaparas #Update parameters (collectively in one big array)
    print('Average weight update size:', np.average(deltaparas))
    #
    #
    a = paras[:L]
    c = paras[L:L+H]
    W = paras[L+H:].reshape(H,L)
    #
    #print('Local Energy: ', ElocalExpVal)
    #
    return W,a,c,ElocalExpVal

#### Main simulation function:

In [4]:
def NQSRBM(J,B,Nv,Nh,kContrastDiv,lrate,epochs):
    # Service message
    print("""\
        Neural Quantum State of the transverse field Ising model:
        Ising model parameters J, B: %f, %f
        Number of visible spins: %i
        Number of hidden spins: %i
        Monte Carlo sequence size: %i
        Learning Rate: %f
        Epochs: %i
        """ %(J,B,Nv,Nh,kContrastDiv,lrate,epochs))
    #
    # Initializing weights with real values between -1 and 1
    # The system is VERY sensitive to initial conditions. 
    # E.g. it will not converge if all weights are negative.
    #
    W0 = (0.2)*(2*np.random.rand(Nh,Nv)-1. +np.random.rand(Nh,Nv)*1j)
    a0 = (0.1)*(2*np.random.rand(Nv)-1. + np.random.rand(Nv)*1j)
    c0 = (0.1)*(2*np.random.rand(Nh)-1. + np.random.rand(Nh)*1j)
    #W0 = np.random.normal(size=(Nh,Nv))/1e4
    #a0 = np.random.normal(size=(Nv))/10
    #c0 = np.random.normal(size=(Nh))/1e4
    W0 = np.real(W0)
    a0 = np.real(a0)
    c0 = np.real(c0)
    #
    # Initialing visible spins with either 0 or 1
    #
    V0 = np.random.choice([0,1],Nv)
    #
    Magnetization = np.sum(V0)-Nv/2
    #while Magnetization > 0:
    #    site = np.random.randint(Nv)
    #    print('Flip-site', site)
    #    if V0[site] > 0 :
    #        V0[site] = - V0[site] + 1 
    #    Magnetization = np.sum(V0)-Nv/2
    #while Magnetization < 0:
    #    site = np.random.randint(Nv)
    #    print('Flip-site', site)
    #    if V0[site] == 0:
    #        V0[site] = - V0[site] + 1 
    #    Magnetization = np.sum(V0)-Nv/2
    #Magnetization = np.sum(V0)-Nv/2
    print('Magnetization Initial state: ', Magnetization)
    #
    # Learning/Variational Minimization cycle
    #
    V = np.copy(V0)
    W = np.copy(W0)
    a = np.copy(a0)
    c = np.copy(c0)
    #
    # The transverse field Ising model happens to
    # be exactly solvable through other means.
    # We secretly know the exact GS energy:
    #
    g = B/J
    FreeFermionModes = np.sqrt(1 + g**2-2*g*np.cos(2*np.pi*np.arange(Nv)/Nv)) 
    EexactPerSite = -J*np.sum(FreeFermionModes)/Nv #Number of modes on each site * energy of occupation = interaction energy
    #
    # Variable Initialization for plotting results
    #
    Convergence = np.array([[1,1]])
    Percentage = np.array([0])
    prct = 0
    #
    for ep in range(epochs):
        #
        Vensemble,prct = MetropolisSamp(W,a,c,V,kContrastDiv) #Get  representative samples
        W,a,c,EExpVal = WeightUpdateSmoothed(J,B,W,a,c,Vensemble,lrate,ep) #Update paramters by fixed paramter gradients on ensemble
        EVarPerSite = np.real(EExpVal)/Nv
        Convergence = np.append(Convergence,np.array([[ep,EVarPerSite]]),axis=0)
        Percentage = np.append(Percentage,np.array([prct]),axis=0)
        #lrate = lrate * 0.95 
        print('\rEpoch %i/%i: Variational Energy: %f, Exact Energy: %f ' %(ep+1,epochs,EVarPerSite,EexactPerSite), end='')
        if not np.abs(EVarPerSite) < 10e6:
            print('\nNumerical Runaway: discontinuing...')
            break
        #print('Weights updated: Started learning epoch %i out of %i\n' %(ep+1,epochs))
    
    WRBM = np.copy(W)
    aRBM = np.copy(a)
    cRBM = np.copy(c)
    sampler = 'MetroSmoothed'
    filename = f'NQSdata_J{J:01}_h{B:01}_{sampler}_Cycles{kContrastDiv}_Epochs{epochs}.pickle'
    print('\nFile = ', filename)
    results = (Convergence, Percentage, aRBM, cRBM, WRBM, EexactPerSite)
    with open(filename,'wb') as f:
        pickle.dump(results,f)
        
    return results     

#### Plotting routines:

In [5]:
import matplotlib.pyplot as plt

def plot_time_independent_convergence(Convergence,Percentage, EexactPerSite):
    Eexc = EexactPerSite*np.ones(Convergence.shape[0]-1)
    fig, ax = plt.subplots()
    
    ax.plot(Convergence[1:,0],Convergence[1:,1], label="Simulated energy per site")
    ax.plot(Convergence[1:,0],Eexc, label="Exact energy per site")
    ax2 = ax.twinx()
    ax2.plot(Convergence[1:,0],Percentage[1:],color='red',linestyle=':', label="Rejection rate")
    ax2.set_ylim(0,100)
    
    ax.set_title('Convergence')
    ax.set_xlabel('Epoch')
    ax.set_ylabel(r'${E_{loc}}/{L}$')
    ax2.set_ylabel("Rejection rate")
    ax.legend()
    fig;
    

#### Example for running a time-independent simulation:

In [8]:

NQSrun = NQSRBM(1.,0.5,10,40,6000,0.4,52)
Convergence,Percentage, aRBM, cRBM, WRBM, EexactPerSite = NQSrun
#
# Displaying analytics
#
plot_time_independent_convergence(Convergence, Percentage, EexactPerSite)

        Neural Quantum State of the transverse field Ising model:
        Ising model parameters J, B: 1.000000, 0.500000
        Number of visible spins: 10
        Number of hidden spins: 40
        Monte Carlo sequence size: 6000
        Learning Rate: 0.400000
        Epochs: 52
        
Magnetization Initial state:  0.0
Percentage Rejections in Burn-in: 27.33 %
Gradient absval-squared: a: 0.0580, c: 0.0006, W: 0.0152. Average weight update size: -0.0001371753147556237
Epoch 1/52: Variational Energy: -0.459173, Exact Energy: -1.063528 Percentage Rejections in Burn-in: 27.83 %
Gradient absval-squared: a: 0.0523, c: 0.0006, W: 0.0137. Average weight update size: -0.0001474435002236764
Epoch 2/52: Variational Energy: -0.464265, Exact Energy: -1.063528 Percentage Rejections in Burn-in: 26.60 %
Gradient absval-squared: a: 0.0501, c: 0.0005, W: 0.0131. Average weight update size: -0.00018169797028390208
Epoch 3/52: Variational Energy: -0.461060, Exact Energy: -1.063528 Percentage Rejecti

KeyboardInterrupt: 